In [1]:
import numpy as np 
import nevis 

h = nevis.gb()

area = np.load('../res/area-with-sea.npy')
label = np.load('../res/label.npy')
maxima = np.load('../res/maxima.npy')
path_sum = np.load('../res/path_sum.npy')

expected_fe = path_sum / area

- `maxima` = [($x_0, y_0$), ... ], where $(x_i, y_i)$ is the grid coordinate of the $i$-th hill, sorted from the highest to the lowest. The length of maxima is the number of hills = number of basin of attraction.
- `label[x][y]` means grid coordinate $(x, y)$ belongs to the $i$-th basin of attraction (which belongs to the $i$-th hill).
- `area[i]` returns the area of the $i$-th basin of attraction (including sea).
- `path_sum[i]` returns the sum of (gradient ascending) path lengths from each point to the $i$-th local max in the $i$-th b.o.a. 
- `path_sum[i] / area[i]` is the average path length (function evaluation) from each point in the $i$-th b.o.a to the $i$-th local max.

In [2]:
# check that the sum of areas add up
m, n = nevis.dimensions()
print(m * n // 2500)
print(int(np.sum(area)))
assert int(np.sum(area)) == m * n // 2500

364000000
364000000


In [3]:
# check that the i-th local max belongs to the i-th b.o.a.
len_maxima = len(maxima)
print(len_maxima)
for i in range(len_maxima):
    assert i == label[maxima[i, 0], maxima[i, 1]]

957174


In [4]:
# a two-line tour to ben nevis
x, y = maxima[0]
h[x-1:x+2, y-1:y+2]

array([[1343.1, 1342.1, 1340.9],
       [1339.6, 1345.3, 1343.8],
       [1222.5, 1299.1, 1327.7]], dtype=float32)

### Calculation of Table 1 & Table 2

In [5]:
intervals = [
    (1340, 1350),
    (1310, 1340),
    (1297, 1310),
    (1235, 1297),
    (1215, 1235),
    (1150, 1215),
    (1100, 1150),
    (1000, 1100),
    (600, 1000),
    (0, 600),
    (-100, 0)
]

In [6]:
from collections import defaultdict

# sum of area of b.o.a. of an interval
interval_area = defaultdict(int)
# sum of path length of b.o.a. of an interval
interval_sum_path = defaultdict(float)
# sum of all areas (check)
s_prime = 0

for i, (x, y) in enumerate(list(maxima)):
    z = h[x, y]

    # find the interval this height belongs to
    for a, b in intervals:
        if a <= z < b:
            break
    else:
        print(z, h[x, y])
    
    interval_area[a] += area[i]
    s_prime += area[i]
    interval_sum_path[a] += path_sum[i]

In [7]:
# check that the sum of areas add up
s_prime

np.int32(364000000)

This is the data for Table 1:

In [8]:
interval_area

defaultdict(int,
            {1340: np.int32(878),
             1310: np.int32(2181),
             1297: np.int32(1322),
             1235: np.int32(11793),
             1215: np.int32(4988),
             1150: np.int32(41414),
             1100: np.int32(66454),
             1000: np.int32(261489),
             600: np.int32(5768408),
             0: np.int32(357718480),
             -100: np.int32(122593)})

In [9]:
interval_sum_path

defaultdict(float,
            {1340: np.float64(23124.0),
             1310: np.float64(73531.0),
             1297: np.float64(24059.0),
             1235: np.float64(242418.0),
             1215: np.float64(109301.0),
             1150: np.float64(936444.0),
             1100: np.float64(1231966.0),
             1000: np.float64(5864836.0),
             600: np.float64(115924225.0),
             0: np.float64(163118008842.0),
             -100: np.float64(502026.0)})

In [10]:
# average function eval for a run which ends up in a local max in this interval
interval_avg_fe = {}
for k in interval_sum_path.keys():
    interval_avg_fe[k] = interval_sum_path[k] / interval_area[k]

interval_avg_fe

{1340: np.float64(26.337129840546698),
 1310: np.float64(33.714351215038974),
 1297: np.float64(18.198940998487142),
 1235: np.float64(20.556092597303486),
 1215: np.float64(21.912790697674417),
 1150: np.float64(22.611773796300767),
 1100: np.float64(18.53862822403467),
 1000: np.float64(22.42861458799414),
 600: np.float64(20.096398347689693),
 0: np.float64(455.9954767838665),
 -100: np.float64(4.095062523961401)}

In [11]:
s = s_prime
# the lower bound for each interval, ordered from low to high
u = [x for x, _ in intervals][::-1]

# convert the dictionary interval_avg_fe to a list
a = [interval_avg_fe[x] for x in u ]

# the probablity of a random initial point to start 
# in an b.o.a. whose local max belongs to each interval
p = [interval_area[x] / s for x in u]

In [12]:
u

[-100, 0, 600, 1000, 1100, 1150, 1215, 1235, 1297, 1310, 1340]

In [13]:
a

[np.float64(4.095062523961401),
 np.float64(455.9954767838665),
 np.float64(20.096398347689693),
 np.float64(22.42861458799414),
 np.float64(18.53862822403467),
 np.float64(22.611773796300767),
 np.float64(21.912790697674417),
 np.float64(20.556092597303486),
 np.float64(18.198940998487142),
 np.float64(33.714351215038974),
 np.float64(26.337129840546698)]

In [14]:
p

[np.float64(0.000336793956043956),
 np.float64(0.9827430769230769),
 np.float64(0.015847274725274724),
 np.float64(0.0007183763736263736),
 np.float64(0.00018256593406593407),
 np.float64(0.00011377472527472527),
 np.float64(1.3703296703296703e-05),
 np.float64(3.239835164835165e-05),
 np.float64(3.6318681318681317e-06),
 np.float64(5.9917582417582414e-06),
 np.float64(2.4120879120879122e-06)]

In [15]:
sum(p) # this should be one

np.float64(1.0)

In [16]:
def calc(p, a, k, n):
    return sum(p[i] * a[i] for i in range(n)) / (1 - sum(p[i] for i in range(k - 1)))
n = len(u) # number of intervals
for k in range(n):
    print(u[k], calc(p, a, k, n))
# this is the data for Table 2

-100 448.46961750549457
0 448.46961750549457
600 448.62071024926263
1000 26505.094275674932
1100 418015.3610246665
1150 1265154.9311940982
1215 2608714.8550877515
1235 7713965.635193093
1297 10092923.257806841
1310 37261570.593777515
1340 53364805.74427704


In [17]:
# convert the data to latex format
def to_latex(float_number):
    import math
    # Extracting the exponent part
    exponent = int(math.floor(math.log10(abs(float_number))))

    # Extracting the mantissa part
    mantissa = float_number / (10 ** exponent)

    # Formatting the float number in the LaTeX style format
    latex_formatted_number = "\\({:.2f} \\times 10^{{{}}}\\)".format(mantissa, exponent)
    return latex_formatted_number

The following is table 2:

In [18]:
n = len(u)
for k in range(n):
    print('\\(', u[k], '\\)', '&', to_latex(calc(p, a, k, n)), '\\\\')

\( -100 \) & \(4.48 \times 10^{2}\) \\
\( 0 \) & \(4.48 \times 10^{2}\) \\
\( 600 \) & \(4.49 \times 10^{2}\) \\
\( 1000 \) & \(2.65 \times 10^{4}\) \\
\( 1100 \) & \(4.18 \times 10^{5}\) \\
\( 1150 \) & \(1.27 \times 10^{6}\) \\
\( 1215 \) & \(2.61 \times 10^{6}\) \\
\( 1235 \) & \(7.71 \times 10^{6}\) \\
\( 1297 \) & \(1.01 \times 10^{7}\) \\
\( 1310 \) & \(3.73 \times 10^{7}\) \\
\( 1340 \) & \(5.34 \times 10^{7}\) \\


The following is table 1:

In [19]:
prefixes = [
    "\([-100, 0)\)     & Below sea-level                                         & ---  &",
    "\([0, 600)\)     & Lowland areas                                           & ---  &",
    "\([600, 1000)\)  & Mountainous areas                                       & ---   &",
    "\([1000, 1100)\) & Mountains within top \(\sim\)135 Munros \& 5 Welsh Furths & ---   &",
    "\([1100, 1150)\) & Mountains within top \(\sim\)50 Munros                    & --- &",
    "\([1150, 1215)\) & Mountains within top \(\sim\)25 Munros                    & ---   &",
    "\([1215, 1235)\) & Wider Ben Nevis Massif (within top 9 Munros)                   & 1     &",
    "\([1235, 1297)\) & Cairngorm Plateau (within top 6 Munros)                        & 2    & ",
    "\([1297, 1310)\) & Ben Macdui (2nd highest Munro)                          & 3     &",
    "\([1310, 1340)\) & On Ben Nevis but not quite at the summit                & 7   &",
    "\([1340, 1345)\) & Ben Nevis (highest Munro)                               & 10         &",
]

for p, (a, c) in zip(prefixes, list(interval_area.items())[::-1]):
    print(p, to_latex(c), '&', to_latex(c / s), '\\\\')

\([-100, 0)\)     & Below sea-level                                         & ---  & \(1.23 \times 10^{5}\) & \(3.37 \times 10^{-4}\) \\
\([0, 600)\)     & Lowland areas                                           & ---  & \(3.58 \times 10^{8}\) & \(9.83 \times 10^{-1}\) \\
\([600, 1000)\)  & Mountainous areas                                       & ---   & \(5.77 \times 10^{6}\) & \(1.58 \times 10^{-2}\) \\
\([1000, 1100)\) & Mountains within top \(\sim\)135 Munros \& 5 Welsh Furths & ---   & \(2.61 \times 10^{5}\) & \(7.18 \times 10^{-4}\) \\
\([1100, 1150)\) & Mountains within top \(\sim\)50 Munros                    & --- & \(6.65 \times 10^{4}\) & \(1.83 \times 10^{-4}\) \\
\([1150, 1215)\) & Mountains within top \(\sim\)25 Munros                    & ---   & \(4.14 \times 10^{4}\) & \(1.14 \times 10^{-4}\) \\
\([1215, 1235)\) & Wider Ben Nevis Massif (within top 9 Munros)                   & 1     & \(4.99 \times 10^{3}\) & \(1.37 \times 10^{-5}\) \\
\([1235, 1297)\) & Cairngorm Pl

### Number of local maxima over 3000 feet

In [20]:
cnt_3000_feet = 0
cnt_all = 0
cnt_3000_feet_north_glasgow = 0

for x, y in maxima:
    z = h[x, y]
    if z >= 914.4:
        cnt_3000_feet += 1
        if x >= 658388 // 50:
            cnt_3000_feet_north_glasgow += 1
    cnt_all += 1

print(cnt_3000_feet)
print(cnt_all)
print(cnt_3000_feet_north_glasgow)

1242
957174
1192


In [21]:
h = nevis.gb()

In [22]:
import nevis
hill = nevis.Hill.by_name('Cathkin Braes') # this place is roughly Glasgow
x, y = hill.coords.grid
h[y // 50, x // 50]

np.float32(199.0)

The coordinate is confusing... you have to swap it and multiply/divide by 50.

In [23]:
x, y

(np.int64(261476), np.int64(658388))

In [24]:
hill = nevis.Hill.by_rank(1)
x1, y1 = hill.coords.grid
x1, y1

(np.int64(216666), np.int64(771288))

In [25]:
h[y1 // 50, x1 // 50]

np.float32(1345.3)

In [26]:
h.shape

(26000, 14000)